In [1]:
from haystack import Pipeline
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack import Document

# Function to read text file
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found. Please make sure the file exists and the path is correct.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {str(e)}")
        return None

# Read the text file
file_path = "1.PMAY-U 2.0 FAQ.txt"  # Assuming you've converted the .docx to .txt
document_content = read_text_file(file_path)

if document_content is None:
    print("Exiting due to file read error.")
    exit(1)

# Initialize QdrantDocumentStore
document_store = QdrantDocumentStore(
    url='http://localhost:6333',
    recreate_index=True,
    return_embedding=True,
    wait_result_from_api=True,
    # Timeout = 5000
)

# Create a Document object
document = Document(content=document_content)

# Initialize and use SentenceTransformersDocumentEmbedder
document_embedder = SentenceTransformersDocumentEmbedder()
document_embedder.warm_up()
documents_with_embeddings = document_embedder.run([document])

# Write document to the document store
document_store.write_documents(documents_with_embeddings.get("documents"), policy=DuplicatePolicy.OVERWRITE)

# Initialize QdrantEmbeddingRetriever
retriever = QdrantEmbeddingRetriever(document_store=document_store)

# Define the prompt template
template = """
Given only the following information from the PMAY-U 2.0 FAQ document, answer the question.
Ignore your own knowledge.
Context:
{% for document in documents %}
 {{ document.content }}
{% endfor %}
Question: {{ query }}
"""

# Set up the pipeline
pipe = Pipeline()
pipe.add_component("text_embedder", SentenceTransformersTextEmbedder())
pipe.add_component("retriever", retriever)
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model="llama3", url="http://localhost:11434/api/generate"))
pipe.connect("text_embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

/home/abhi/Documents/ChatBot/Llama3/letest_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/abhi/Documents/ChatBot/Llama3/letest_env/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


TypeError: QdrantDocumentStore.__init__() got an unexpected keyword argument 'Timeout'

In [ ]:
# Example query
query = "How Central Assistance will be released for Private Sector AHP Projects ?"
response = pipe.run({"prompt_builder": {"query": query}, "text_embedder": {"text": query}})
print(response["llm"]["replies"])

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.34it/s]


ReadTimeout: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120)